In [ ]:
from pathlib import Path
import duckdb

In [ ]:
# Compute the database path relative to this notebook using cwd
notebook_dir = Path.cwd()
# db_path = (notebook_dir.parent / "database" / "geocoder.duckdb").resolve()
db_path = (notebook_dir.parent / "database" / "degauss.duckdb").resolve()
# con = duckdb.connect(str(db_path))

print(f"Connected to {db_path}")

In [ ]:
# List all available tables (excluding per-county tables like tl_2021_13001_*)
with duckdb.connect(str(db_path)) as conn:
    tables = conn.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'main' 
        AND table_name NOT LIKE 'tl_%' -- exludes per-county tables
        ORDER BY table_name
    """).fetchall()
    tbl_ls = [table[0] for table in tables]
    print(f"Available consolidated tables ({len(tables)}):")
    for table in tables:
        print(f"  - {table[0]}")
    print(tbl_ls)

In [ ]:
# Load data from all consolidated tables
dataframes = {}
with duckdb.connect(str(db_path)) as conn:
    for tbl in tbl_ls:
        df = conn.execute(f"SELECT * FROM {tbl} LIMIT 100").df()
        dataframes[tbl] = df
        print(f"Loaded {len(df)} rows from {tbl}")

In [ ]:
# Display dataframes in tabbed interface
from ipywidgets import Tab, Output
from IPython.display import display

# Create output widgets for each table
outputs = []
for tbl_name, df in dataframes.items():
    out = Output()
    with out:
        print(f"Table: {tbl_name} ({len(df)} rows, {len(df.columns)} columns)")
        display(df)
    outputs.append(out)

# Create tab widget
tab = Tab(children=outputs)
for i, tbl_name in enumerate(dataframes.keys()):
    tab.set_title(i, tbl_name)

display(tab)

In [ ]:
## Convert to DeGAUSS Format

Convert the TIGER/Line tables to DeGAUSS geocoding format with optimized schema and pre-computed metaphone codes.

## Verify DeGAUSS Database

In [ ]:
# Connect to the new DeGAUSS database and verify tables
degauss_db = str(db_path.parent / "degauss.duckdb")

with duckdb.connect(degauss_db) as conn:
    # Load spatial extension
    conn.execute("INSTALL spatial;")
    conn.execute("LOAD spatial;")
    
    # Get table info
    tables = conn.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'main'").fetchall()
    print(f"DeGAUSS Database Tables:")
    for table in tables:
        count = conn.execute(f"SELECT COUNT(*) FROM {table[0]}").fetchone()[0]
        print(f"  - {table[0]}: {count:,} rows")
    
    print("\nSample geocoder record:")
    sample = conn.execute("SELECT tlid, fullname, name_metaphone, zipl, lat, lon FROM geocoder LIMIT 5").df()
    display(sample)
    
    print("\nSample street_names record:")
    names_sample = conn.execute("SELECT tlid, fullname, name, name_metaphone FROM street_names LIMIT 5").df()
    display(names_sample)